# 只包含 u 和 cz 门

In [27]:
import numpy as np
from qiskit import QuantumCircuit, qasm2
import math
import re

# 预定义π的数值常量（使用15位精度）
PI_VAL = 3.141592653589793
HALF_PI = PI_VAL / 2  # 1.5707963267948966
QUARTER_PI = PI_VAL / 4  # 0.7853981633974483
TWO_PI = 2 * PI_VAL  # 6.283185307179586

def calculate_expression(expr):
    """安全计算表达式并返回浮点数值"""
    # 替换所有π相关符号为数值
    expr = expr.replace("pi", str(PI_VAL))
    expr = expr.replace("π", str(PI_VAL))
    
    # 计算表达式
    try:
        return float(eval(expr, {"__builtins__": None}, {"pi": PI_VAL}))
    except:
        # 如果无法计算，尝试直接转换
        try:
            return float(expr)
        except:
            raise ValueError(f"无法解析表达式: {expr}")

def convert_qasm_to_numeric(qasm_path):
    """
    将QASM文件中的所有角度转换为数值（避免pi符号和表达式）
    返回仅包含数值角度的新QASM字符串
    """
    with open(qasm_path, 'r') as f:
        content = f.read()
    
    # 使用正则表达式匹配u门的所有参数
    pattern = r'u\(([^)]+)\)'
    matches = re.findall(pattern, content)
    
    # 替换所有表达式为计算后的数值
    for match in matches:
        params = [p.strip() for p in match.split(',')]
        numeric_params = []
        
        # 计算每个参数
        for param in params:
            # 尝试直接转换为浮点数
            try:
                numeric_val = float(param)
            except ValueError:
                # 如果包含表达式，计算它
                numeric_val = calculate_expression(param)
            
            numeric_params.append(str(numeric_val))
        
        # 创建新的u门指令
        new_u = f"u({','.join(numeric_params)})"
        
        # 替换原内容
        old_u = f"u({match})"
        content = content.replace(old_u, new_u)
    
    # 创建临时电路来解析并标准化
    temp_circuit = QuantumCircuit.from_qasm_str(content)
    
    # 创建新电路并添加所有操作（确保使用数值）
    new_circuit = QuantumCircuit(temp_circuit.num_qubits)
    for instruction in temp_circuit.data:
        op = instruction.operation
        qubits = instruction.qubits
        
        # 只处理u和cz门
        if op.name == 'u':
            # 确保所有参数都是数值
            params = [float(p) for p in op.params]
            new_circuit.u(params[0], params[1], params[2], *qubits)
        elif op.name == 'cz':
            new_circuit.cz(*qubits)
        elif op.name == 'barrier':
            new_circuit.barrier(qubits)
        else:
            raise ValueError(f"初始电路包含不支持的门: {op.name}")
    
    return qasm2.dumps(new_circuit)

def apply_qft(qc, qubits):
    """手动实现QFT（仅使用u和cz门，使用数值角度）"""
    n = len(qubits)
    # 正向QFT
    for j in range(n):
        # Hadamard用u门实现: u(π/2, 0, π) -> 使用数值
        qc.u(HALF_PI, 0.0, PI_VAL, qubits[j])
        
        # 控制相位门
        for k in range(j+1, n):
            exponent = k - j
            angle = PI_VAL / (2**exponent)  # 数值计算
            
            # 控制相位门分解为u门和cz门
            qc.u(0.0, 0.0, angle/2, qubits[k])
            qc.cz(qubits[j], qubits[k])
            qc.u(0.0, 0.0, -angle/2, qubits[k])
            qc.cz(qubits[j], qubits[k])
            qc.u(0.0, 0.0, angle/2, qubits[k])
    
    # 交换操作（如果需要）
    for i in range(n//2):
        a, b = qubits[i], qubits[n-i-1]
        qc.cz(a, b)
        qc.u(HALF_PI, 0.0, PI_VAL, a)  # H
        qc.u(HALF_PI, 0.0, PI_VAL, b)  # H
        qc.cz(a, b)
        qc.u(HALF_PI, 0.0, PI_VAL, a)  # H
        qc.u(HALF_PI, 0.0, PI_VAL, b)  # H
        qc.cz(a, b)

def apply_iqft(qc, qubits):
    """手动实现IQFT（仅使用u和cz门，使用数值角度）"""
    n = len(qubits)
    # 反向交换操作
    for i in range(n//2):
        a, b = qubits[i], qubits[n-i-1]
        qc.cz(a, b)
        qc.u(HALF_PI, 0.0, PI_VAL, a)  # H
        qc.u(HALF_PI, 0.0, PI_VAL, b)  # H
        qc.cz(a, b)
        qc.u(HALF_PI, 0.0, PI_VAL, a)  # H
        qc.u(HALF_PI, 0.0, PI_VAL, b)  # H
        qc.cz(a, b)
    
    # 逆相位门
    for j in range(n-1, -1, -1):
        for k in range(n-1, j, -1):
            exponent = k - j
            angle = -PI_VAL / (2**exponent)  # 负号表示逆操作，数值计算
            
            # 控制相位门分解
            qc.u(0.0, 0.0, angle/2, qubits[k])
            qc.cz(qubits[j], qubits[k])
            qc.u(0.0, 0.0, -angle/2, qubits[k])
            qc.cz(qubits[j], qubits[k])
            qc.u(0.0, 0.0, angle/2, qubits[k])
        
        # Hadamard逆操作: u(π/2, π, π) = H†
        qc.u(HALF_PI, PI_VAL, PI_VAL, qubits[j])

def generate_full_evolution_circuit(initial_state_qasm_path, t, n_x, n_y):
    """
    生成完整的时间演化电路（仅使用u和cz门，使用数值角度）
    
    参数:
        initial_state_qasm_path: 初态制备的QASM文件路径
        t: 演化时间
        n_x: x方向量子比特数
        n_y: y方向量子比特数
        
    返回:
        完整量子电路 (QuantumCircuit)
    """
    n_total = n_x + n_y
    
    # 转换初始QASM为纯数值版本
    initial_qasm = convert_qasm_to_numeric(initial_state_qasm_path)
    initial_circuit = QuantumCircuit.from_qasm_str(initial_qasm)
    
    # 验证量子比特数
    if initial_circuit.num_qubits != n_total:
        raise ValueError(f"初态电路的量子比特数({initial_circuit.num_qubits})与系统大小({n_total})不匹配")
    
    # 创建完整电路
    full_circuit = QuantumCircuit(n_total)
    
    # 添加初态制备电路（已转换为数值）
    full_circuit.compose(initial_circuit, inplace=True)
    
    # 添加屏障标记
    full_circuit.barrier(range(n_total))
    
    # 核心演化结构（仅使用u和cz门，使用数值角度）
    def add_direction_evolution(qc, qubits, t, direction_size):
        n_qubits = len(qubits)
        
        # 应用QFT（手动实现）
        apply_qft(qc, qubits)
        
        # 添加相位旋转（使用u门实现rz，使用数值角度）
        # 关键修正：根据公式四，Z_1前的系数应为2^{n_α - 1}
        for i, q in enumerate(qubits):
            # 计算指数：第一个量子比特(MSB)对应2^{n_α-1}，最后一个(LSB)对应2^0
            exponent = direction_size - 1 - i  # 修正：方向大小应为n_x或n_y
            
            # 系数计算：2^{exponent}
            coefficient = 2 ** exponent
            
            # 角度计算：-t * coefficient * π
            angle = -t * coefficient * PI_VAL  # 数值计算
            qc.u(0.0, 0.0, angle, q)
        
        # 应用逆QFT（手动实现）
        apply_iqft(qc, qubits)
    
    # 为x方向添加演化
    x_qubits = list(range(n_x))
    add_direction_evolution(full_circuit, x_qubits, t, n_x)
    
    # 为y方向添加演化
    y_qubits = list(range(n_x, n_x + n_y))
    add_direction_evolution(full_circuit, y_qubits, t, n_y)
    
    return full_circuit

def generate_full_qasm(initial_state_qasm_path, t, n_x, n_y, output_file="full_evolution.qasm"):
    """
    生成完整电路的QASM（仅使用u和cz门，使用数值角度）
    
    参数:
        initial_state_qasm_path: 初态制备的QASM文件路径
        t: 演化时间
        n_x: x方向量子比特数
        n_y: y方向量子比特数
        output_file: 输出文件名
        
    返回:
        QASM字符串
    """
    # 生成完整电路
    full_circuit = generate_full_evolution_circuit(initial_state_qasm_path, t, n_x, n_y)
    
    # 直接生成QASM字符串
    qasm_str = qasm2.dumps(full_circuit)
    
    # 保存文件
    with open(output_file, 'w') as f:
        f.write(qasm_str)
    
    return qasm_str, full_circuit




In [28]:

# 使用示例
if __name__ == "__main__":
    # 参数设置
    initial_qasm_file = "diverging_flow_t=0.qasm"  # 替换为您的QASM文件路径
    output_qasm_file = "diverging_flow_t=pi_over_2_test_2.qasm"
    
    # 使用数值π值
    t_value = PI_VAL / 2  # 演化时间
    
    n_x, n_y = 5, 5  # 系统尺寸
    
    # 生成完整QASM
    qasm_output, full_circuit = generate_full_qasm(
        initial_state_qasm_path=initial_qasm_file,
        t=t_value,
        n_x=n_x,
        n_y=n_y,
        output_file=output_qasm_file
    )
    
    print(f"已生成完整电路QASM，保存至{output_qasm_file}")
    print(f"总量子比特数: {n_x + n_y}")
    print(f"电路深度: {full_circuit.depth()}")
    print(f"门计数: {full_circuit.count_ops()}")
    
    # 严格验证QASM中无"pi"符号和表达式
    with open(output_qasm_file, 'r') as f:
        content = f.read()
        if "pi" in content.lower() or "/" in content:
            # 如果发现pi符号或除号，进行二次清理
            print("检测到残留'pi'符号或表达式，进行二次清理...")
            
            # 使用更严格的转换
            with open(output_qasm_file, 'r') as f_in:
                content = f_in.read()
                
            # 替换所有pi相关符号
            content = content.replace("pi", str(PI_VAL))
            content = content.replace("π", str(PI_VAL))
            
            # 计算所有表达式
            pattern = r'u\(([^)]+)\)'
            matches = re.findall(pattern, content)
            
            for match in matches:
                params = [p.strip() for p in match.split(',')]
                numeric_params = []
                
                for param in params:
                    try:
                        # 尝试计算表达式
                        numeric_val = calculate_expression(param)
                        numeric_params.append(str(numeric_val))
                    except:
                        # 如果无法计算，保留原值
                        numeric_params.append(param)
                
                new_u = f"u({','.join(numeric_params)})"
                old_u = f"u({match})"
                content = content.replace(old_u, new_u)
            
            # 保存清理后的文件
            with open(output_qasm_file, 'w') as f_out:
                f_out.write(content)
            
            print("已完成二次清理")
        else:
            print("验证通过：QASM文件中无'pi'符号或表达式")
    
    # 最终验证
    with open(output_qasm_file, 'r') as f:
        final_content = f.read().lower()
        if "pi" in final_content or "/" in final_content:
            print("错误：最终QASM文件中仍存在'pi'符号或表达式")
            # 打印问题行
            for i, line in enumerate(final_content.split('\n')):
                if "pi" in line or "/" in line:
                    print(f"问题行 {i+1}: {line.strip()}")
        else:
            print("最终验证通过：QASM文件中无'pi'符号或表达式")

已生成完整电路QASM，保存至diverging_flow_t=pi_over_2_test_2.qasm
总量子比特数: 10
电路深度: 84
门计数: OrderedDict([('u', 209), ('cz', 113), ('barrier', 1)])
检测到残留'pi'符号或表达式，进行二次清理...
已完成二次清理
最终验证通过：QASM文件中无'pi'符号或表达式
